In [58]:
train.shape

(5028, 905)

In [25]:
# coding: utf-8
# pylint: disable = invalid-name, C0111
import json
import lightgbm as lgb
import pandas as pd
import numpy
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score

# load or create your dataset
print('Load data...')
fulldataset = pd.read_csv('train.csv',sep=';')
columns = ['cash_in_out','display_type','scanner_code_reader']
fulldataset.drop(columns, inplace=True, axis=1)

test2 = pd.read_csv('test.csv', sep=';')
test2.drop(columns, inplace=True, axis=1)
test2 = test2.values

train, test = train_test_split(fulldataset, test_size=0.1)

target = train['target'].values
target_test = test['target'].values

#df_train2 = df_train[['cardreader_error_by_day','status_count']]
#columns = ['cash_in_out','display_type','scanner_code_reader']
#train.drop(columns, inplace=True, axis=1)
#train = train.values
train = train[['cardreader_error_by_day','status_count']].values
#train = lgb.Dataset('train.svm.txt')
#train.save_binary('train.bin')

#df_test2 = df_test[['cardreader_error_by_day','status_count']]
#test.drop(columns, inplace=True, axis=1)
#test = test.values
test = test[['cardreader_error_by_day','status_count']].values
# create dataset for lightgbm
lgb_train = lgb.Dataset(train, target)
lgb_eval = lgb.Dataset(test, target_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': {'auc'},
    'num_leaves': 5,
    'learning_rate': 0.03,
    'depth':3,
        'n_estimators': 22,
        'min_data_in_leaf': 60,
        'col_sample_rate' : 0.3
}
print('Start training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

#gbm = lgb.train(params, lgb_train, num_boost_round=20)
print('Save model...')
# save model to file
gbm.save_model('model.txt')
#print('Start predicting...')
# predict
PREDICT = gbm.predict(test, num_iteration=gbm.best_iteration)

# eval
PREDICT[PREDICT > 0.5] = 1
PREDICT[PREDICT <= 0.5] = 0
print('Validaton:', f1_score(target_test, PREDICT))


Load data...
Start training...
[1]	valid_0's auc: 0.547236
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.547236
[3]	valid_0's auc: 0.546767
[4]	valid_0's auc: 0.546767
[5]	valid_0's auc: 0.554672
[6]	valid_0's auc: 0.554672
[7]	valid_0's auc: 0.552994
[8]	valid_0's auc: 0.552994
[9]	valid_0's auc: 0.552994
[10]	valid_0's auc: 0.552994
Early stopping, best iteration is:
[5]	valid_0's auc: 0.554672
Save model...
Validaton: 0.523178807947


/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [12]:
PREDICT[0:10]

array([ 0.47973445,  0.47973445,  0.50116737,  0.50562173,  0.46341827,
        0.49353523,  0.50116737,  0.47973445,  0.48686549,  0.51009448])

In [189]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
final_col_names = fulldataset.columns
fulldataset = fulldataset.values
fulldataset[:, cont_feats_idx] = scaler.fit_transform(fulldataset[:, cont_feats_idx])

scaler_2 = MinMaxScaler(feature_range=(0, 1))
fulldataset[:, int_feats_idx] = scaler_2.fit_transform(fulldataset[:, int_feats_idx])

NameError: name 'cont_feats_idx' is not defined

NameError: name 'mean' is not defined

In [23]:
mean(PREDICT)

NameError: name 'mean' is not defined